In [49]:
# imports
import pandas as pd
import requests
import pickle as p
import matplotlib.pyplot as plt

In [2]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

### Mapping scratch pad:

helpful Tableau tutorial video:  https://www.youtube.com/watch?v=tr51B8ZKAq8
- geometry:field, name:detail, data:color

In [50]:
#stack
# !pip install simpledbf
import simpledbf
from simpledbf import Dbf5

In [51]:
#look at geo files from SWDB and see what's in there:
dbf = Dbf5('./map_files/2011_congressional_state_equiv.dbf')
df = dbf.to_dataframe()

In [10]:
df.head()

,BLOCK,DISTRICTID
0,060650432722072,42
1,060650432722026,42
2,060650432722070,42
3,060650432722071,42
4,060650432722025,42


In [11]:
df.shape

(710144, 2)

In [12]:
df.groupby('DISTRICTID').count().tail()
#53 congressional districts in 2011

,BLOCK
DISTRICTID,
49,9993
50,13137
51,15763
52,8717
53,7899


In [13]:
x = df.loc[df['DISTRICTID'] == '53']
x[:10]

,BLOCK,DISTRICTID
10016,060730001001016,53
10017,060730001001006,53
10018,060730001001000,53
10019,060730001001017,53
10020,060730001001018,53
10021,060730001001030,53
10022,060730001001008,53
10023,060730001001004,53
10024,060730001001001,53
10025,060730001001003,53


In [14]:
x.shape   #7899 blocks in Dist53

(7899, 2)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710144 entries, 0 to 710143
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   BLOCK       710144 non-null  object
 1   DISTRICTID  710144 non-null  object
dtypes: object(2)
memory usage: 10.8+ MB


In [16]:
# pandas documentation
# s.str.slice(start=0, stop=5, step=3)
# s.stri.slice[0:5:3]
df['state'] = df['BLOCK'].str.slice(stop=2)
df['county'] = df['BLOCK'].str.slice(start=2, stop=5)
df['tract'] = df['BLOCK'].str.slice(start=5, stop=11)
df['block'] = df['BLOCK'].str.slice(start=11, stop=15)

In [48]:
w = df.loc[df['DISTRICTID'] == '39']
w.shape

(8256, 6)

In [46]:
w[:10]
# df.drop(columns=['tracts'])

,BLOCK,DISTRICTID,state,county,tract,block
64656,060590219241056,39,06,059,021924,1056
64657,060590219241055,39,06,059,021924,1055
64658,060590219241022,39,06,059,021924,1022
64659,060590218261014,39,06,059,021826,1014
64660,060590218261010,39,06,059,021826,1010
64661,060590218283006,39,06,059,021828,3006
64662,060590218261005,39,06,059,021826,1005
64663,060590218261006,39,06,059,021826,1006
64664,060590218261007,39,06,059,021826,1007
64665,060590218261009,39,06,059,021826,1009


In [47]:
w['county'].value_counts(normalize=True)

059    0.663639
037    0.249637
071    0.086725
Name: county, dtype: float64

In [20]:
congress_dist = df.loc[df['county'] == '065'].nunique()
congress_dist

BLOCK         35718
DISTRICTID        4
state             1
county            1
tract           453
block          2003
dtype: int64

### Srprec <> Census block/tract -- explore

In [54]:
xlate = pd.read_csv('./map_files/c059_g12_sr_blk_map.csv')
xlate.shape

(23490, 8)

In [55]:
xlate.tail()

,srprec,tract,block,blkreg,srtotreg,pctsrprec,blktotreg,pctblk
23485,75602,75809,2007,604,1437,42.032011,613,98.531811
23486,75602,75814,2003,14,1437,0.974252,14,100.000000
23487,75602,75814,2016,2,1437,0.139179,2,100.000000
23488,75602,75814,2017,28,1437,1.948504,28,100.000000
23489,75901,75809,2007,9,9,100.000000,613,1.468189


In [56]:
xlate['tract'].nunique()
#582 census tracts in g12 data set.

582

In [57]:
xlate['srprec'].nunique()
#1877 srprec's

1877

In [58]:
xlate['block'].nunique()

558

In [59]:
xlate['tract'].value_counts()

63500    235
62800    178
62702    141
52420    138
62604    132
        ... 
63008      6
99502      6
21813      5
52518      4
52404      2
Name: tract, Length: 582, dtype: int64

In [60]:
xlate['block'].value_counts()

2001    468
2002    461
2003    449
2004    434
2000    423
       ... 
7015      1
9008      1
2165      1
2133      1
6053      1
Name: block, Length: 558, dtype: int64

In [61]:
xlate['srprec'].value_counts()

2193     74
53355    58
68296    57
13377    56
68098    55
         ..
2434      1
69902     1
5379      1
44123     1
56383     1
Name: srprec, Length: 1877, dtype: int64

In [66]:
xlate['block'].max()

9021